In this notebook I clean and aggregate equity data to end-of-month data.

I also fetch end of month data for the Russell 3000, proxied by `IWV`, an iShares Russell 3000 ETF.

In [2]:
import pandas as pd
import yfinance as yf
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

load_dotenv('../.env')
db_url = os.getenv('DB_URL')
psql = create_engine(db_url)

First pull the Russell 3K data

In [4]:
iwv = yf.Ticker("IWB")
hist = iwv.history(period='max').reset_index()

iwv_hist = hist[['Date', 'Close', 'Volume']].rename(columns={'Date': 'date', 'Close': 'price', 'Volume': 'volume'})
iwv_hist['ticker'] = 'IWB'
iwv_monthly = iwv_hist.set_index('date').resample('ME').last().reset_index()
iwv_monthly['date'] = iwv_monthly['date'].dt.date

# be careful of re-running this if you already have this saved to the database
iwv_monthly.to_sql('eom_prices', psql, if_exists='append', index=False)

# now we will have to be careful about pulling in data for portfolio construction, to ensure
# that we are not trading the ETF of our benchmark itself...

# i mean maybe this is fine? but i dont see why we would trade anything that is in direct 
# correlation with the market (with some tracking error.)

305

Now aggregate the per-stock equity prices, into end-of-month prices with end-of-month volumes.

We might end up having some volume-informed factors and portfolio construction, particularly because some tickers (even if they have amazing alpha), but not be able to be traded.

That is, having some volume profiler as a constraint is probably necessary.

In [17]:
# you will obviously not be able to run this externally, as these files are stored in
# my personal computer. read the disclaimer about data if you are confused about why
# i am not storing the pricing data on the repository.

prices = pd.read_csv('~/Downloads/prices.csv')

In [19]:
# sort by date and ticker
prices = prices.sort_values(['ticker', 'date'])

In [21]:
# drop any null
prices = prices.dropna()

The below might be really intensive (if anyone ends up getting the Sharadar data to re-create this) because it's doing some heavy transformations on ~4GB of daily equity data.

My 16GB M1 Macbook Air struggled on this, so if you're on 8GB you might need to partition the data into multiple chunks and do it by batch.

In [24]:
# get end-of-month data per ticker
prices['date'] = pd.to_datetime(prices['date'])
monthly_prices = prices.set_index('date').groupby('ticker', group_keys=False).resample('ME').last().reset_index()

/var/folders/qq/9n_h2x6x0pl9pvyr0gpk4m_r0000gn/T/ipykernel_44816/4061356084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['date'] = pd.to_datetime(prices['date'])


In [ ]:
monthly_prices = monthly_prices[['date', 'ticker', 'closeadj', 'volume']].rename(
    columns = {
        'closeadj': 'price'
    }
)

In [40]:
monthly_prices = monthly_prices[monthly_prices['ticker'] != 'BINI']

In [41]:
monthly_prices = monthly_prices.dropna()
monthly_prices.sort_values('price')
monthly_prices['volume'] = monthly_prices['volume'].astype(int)

In [43]:
monthly_prices.sort_values('price')

,date,ticker,price,volume
485217,2017-10-31,CPSL1,1.000000e-04,0
844440,2010-12-31,GIGIQ,1.000000e-04,13400
1800102,2025-07-31,SMXWW,1.000000e-04,18500
1877825,2022-12-31,SWETW,1.000000e-04,51927
1950311,2022-10-31,TMPMW,1.000000e-04,3358
...,...,...,...,...
1958801,2005-03-31,TOPS,4.611881e+14,0
1958813,2006-03-31,TOPS,4.683842e+14,0
170379,2007-12-31,ASTI,4.972000e+14,0
1958800,2005-02-28,TOPS,5.302477e+14,0


In [44]:
monthly_prices.to_sql('eom_prices', psql, if_exists='append', index=False)

233